# Neural networks from scratch with NumPy


Consider a classification problem with $K$ classes and input vector $\mathbf{x}$. Neural networks can be used as discriminative models for the probability $p(\mathcal{C}_i\mid \mathbf{x}, \boldsymbol{\theta})$ of a target class $\mathcal{C}_i$ given the input vector $\mathbf{x}$, where $\mathbf{\Theta}$ denotes all the parameters of the network. In a neural network with $L$ layers, this probability is modeled in the last layer using the softmax function:

$$
p(\mathcal{C}_i\mid \mathbf{x}, \mathbf{\theta}) = a_{iL} = f(z_{iL}) = \frac{\exp(z_{iL})}{\sum_{j=1}^K\exp(z_{jL})}
$$

Here $f(\cdot)$ correspond to the so-called *activation function*, which in this case is the softmax.

We can put all the probabilities $a_{iL}$ (with $i = 1, ..., K$) in a column vector, so that the complete output of the network is a vector $\mathbf{a}_L$ with $K$ elements, containing probabilities for each class. In general, we will call $\mathbf{a}_l$ the activation output of the $l$-th layer.

Note that there is one value of $z_{iL}$ for $i = 1, ..., K$. Each one corresponds to a "unit" in the output layer, and is the result of a linear combination of the activations of the previous layer $\mathbf{a}_{L-1}$ via a set of weights $\mathbf{w}_{iL}$ and a bias (constant term) $b_{iL}$:

$$
z_{iL} = \mathbf{w}_{iL}^T\mathbf{a}_{L-1} + b_{iL}
$$

The weights $\mathbf{w}_{iL}$ are column vectors with a number of elements equal to the number of output units in the previous layer. If we put these in the rows of a matrix $\mathbf{W}_L$, and all the biases in a column vector $\mathbf{b}_L$, we can write all the $z_{iL}$ in a single column vector $\mathbf{z}_L$ given by

$$
\mathbf{z}_L = \mathbf{W}_L\mathbf{a}_{L-1} + \mathbf{b}_L
$$

These equations show how the neural network builds a function composition, from inputs to outputs, in order to model the class probabilities. We can think of the input vector $\mathbf{x}$ as the activation of layer zero, $\mathbf{a}_{0}$, which allows us to write down the equations to perform a forward pass on the neural network:

$$
\begin{align}
\mathbf{z}_1 &= \mathbf{W}_1\mathbf{x} + \mathbf{b}_1\\
\mathbf{a}_1 &= f_1(\mathbf{z}_1)\\
\mathbf{z}_2 &= \mathbf{W}_2\mathbf{a}_1 + \mathbf{b}_2\\
\mathbf{a}_2 &= f_2(\mathbf{z}_2)\\
&\vdots\\
\mathbf{z}_L &= \mathbf{W}_L\mathbf{a}_{L-1} + \mathbf{b}_L\\
\mathbf{a}_L &= f_L(\mathbf{z}_L)
\end{align}
$$

When we say that we "train" the neural network, we aim to find the right weights and biases (which we have collectively called $\boldsymbol{\theta}$) in order to minimize a certain loss function. If we have a dataset $\mathbf{D}$ of input vectors and their corresponding class labels, we can compute the likelihood $p(\mathbf{D}\mid\boldsymbol{\theta})$ of observing the dataset as a function of a particular value of the parameters, which is known as the *likelihood function*, and obtain the set of parameters that maximize it. By doing so, we hope to obtain parameters that not only fit well to the observed data but also generalize to unseen data.

For mathematical convenience (such as converting products into sums that simplify differentiation), it is often preferred to work with the logarithm of the likelihood. If we define the task in terms of minimization, we end up with the goal of finding the parameters that minimize the negative log-likelihood<sup>[1](#1)</sup>:

$$
\arg\min_{\boldsymbol{\theta}}\lbrace-\log p(\mathbf{D}\mid\boldsymbol{\theta})\rbrace
$$

We will solve this optimization problem using gradient descent, by iteratively adjusting the parameters in the direction of decreasing loss. We obtain this direction by calculating the gradient with respect to the parameters.

### Deriving the gradients

To specify the label for an input vector we will use a vector with a one hot encoding, so that the vector has $K$ elements, all zero, except for the position of the correct label, which is one. This way, given an observed target vector $\mathbf{t}$ for an input vector $\mathbf{x}$, the negative log-likelihood is

$$
\begin{align}
L(\boldsymbol{\theta}) = -\log p(\mathbf{t}|\mathbf{x}, \boldsymbol{\theta}) &= -\log\prod_{i=1}^K p(\mathcal{C}_i\mid \mathbf{x}, \boldsymbol{\theta})^{t_i}\\
&= -\sum_{i=1}^K t_i\log p(\mathcal{C}_i\mid \mathbf{x}, \boldsymbol{\theta})\\
&= -\sum_{i=1}^K t_i\log\left(
\frac{\exp(z_{iL})}{\sum_{j=1}^K\exp(z_{jL})}
\right)\\
&= -\sum_{i=1}^K t_i\left(
z_{iL} - \log\sum_{j=1}^K\exp(z_{jL}))
\right)\\
&= z_{mL} - \log\sum_{j=1}^K\exp(z_{jL})\\
&= z_{mL} - \log Z
\end{align}
$$

Where we have defined

$$
Z = \sum_{j=1}^K\exp(z_{jL})
$$

We now have to calculate the gradients of the loss with respect to every parameter of the neural network. This process consists of a careful application of the chain rule, as we will now see. Starting from the last layer and the weights of the $i$-th unit, we have

$$
\frac{\partial L}{\partial\mathbf{w}_{iL}} = \frac{\partial L}{\partial z_{iL}}\frac{\partial z_{iL}}{\partial\mathbf{w}_{iL}}
$$

For the first derivative in this expression we have

$$
\begin{align}
\frac{\partial L}{\partial z_{iL}} &= \frac{\partial}{\partial z_{iL}}[z_{mL} - \log Z]\\
&= \frac{\partial}{\partial z_{iL}}[z_{mL}] - \frac{\partial}{\partial z_{iL}}[\log Z]\\
&= \frac{\partial}{\partial z_{iL}}[z_{mL}] - \frac{1}{Z}\frac{\partial}{\partial z_{iL}}[Z]\\
&= \frac{\partial}{\partial z_{iL}}[z_{mL}] - \frac{1}{Z}\frac{\partial}{\partial z_{iL}}
\left[
\sum_{j=1}^K\exp(z_{jL})
\right]\\
&= \frac{\partial}{\partial z_{iL}}[z_{mL}] - \frac{1}{Z}\frac{\partial}{\partial z_{iL}}[\exp(z_{iL})]\\
&= \frac{\partial}{\partial z_{iL}}[z_{mL}] - \frac{1}{Z}\exp(z_{iL})\\
&= \left\{
\begin{matrix}
1 - \frac{1}{Z}\exp(z_{iL}) \text{ if } i = m\\
- \frac{1}{Z}\exp(z_{iL}) \text{ if } i \neq m
\end{matrix}
\right.
\end{align}
$$

The next derivative is computed as follows:

$$
\begin{align}
\frac{\partial z_{iL}}{\partial\mathbf{w}_{iL}} &= \frac{\partial}{\partial \mathbf{w}_{iL}}[\mathbf{w}_{iL}^T\mathbf{a}_{L-1} + b_{iL}] = \mathbf{a}_{L-1}
\end{align}
$$

We therefore have

$$
\frac{\partial L}{\partial\mathbf{w}_{iL}} = \frac{\partial L}{\partial z_{iL}}\mathbf{a}_{L-1}
$$

This means that for all weights $\mathbf{w}_{iL}$, the gradient is obtained by multiplying the activation vector of the previous layer by the derivative $\partial L/\partial z_{iL}$ (a scalar). The result is a new vector with the same number of elements of $\mathbf{w}_{iL}$. If we put all these in the rows of a matrix, we end up with a gradient matrix of the same size of $\mathbf{W}_{L}$. Furthermore, if we put the derivatives $\partial L/\partial z_{iL}$, for $i=1,...,K$, in a column vector $\partial L/\partial\mathbf{z}_L$, then the gradient of the loss with respect to the weights $\mathbf{W}_{L}$ can be calculated in a compact way using an outer product of vectors:

$$
\nabla_\mathbf{W}L = \frac{\partial L}{\partial\mathbf{z}_L} \mathbf{a}_{L-1}^T =
\begin{pmatrix}
\mid\\ 
\partial L/\partial\mathbf{z}_L\\
\mid
\end{pmatrix}
\begin{pmatrix}
-\;\mathbf{a}_{L-1}\;-
\end{pmatrix}
=
\begin{pmatrix}
-\;\frac{\partial L}{\partial\mathbf{w}_{1L}}\;-\\
-\;\frac{\partial L}{\partial\mathbf{w}_{2L}}\;-\\
\vdots\\
-\;\frac{\partial L}{\partial\mathbf{w}_{KL}}\; -
\end{pmatrix}
$$

<a name="1">1</a>. In practice we don't necessarily want to find the minimum for this objective. We could minimize it as much as we can by choosing an appropriately complex model and training for long enough, but we might end up overfitting to the training set and with a model that does not generalize well to unseen data. In the end, we want our model to perform well both on training and test data, and with neural networks there are multiple ways to achieve this, such as weight regularization, dropout, data augmentation, among other techniques.